In [1]:
import gymnasium as gym 
from gymnasium import Env
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import tensorflow as tf
import datetime
import gymnasium as gym
import numpy as np
from stable_baselines3 import DQN
from stable_baselines3.common.env_util import make_vec_env
import tensorflow as tf

# # Set seed for reproducibility
# seed = 42
# random.seed(seed)
# np.random.seed(seed)

class missile_interception(Env):
    def __init__(self):
        self.action_space = gym.spaces.discrete.Discrete(3)

        """STATE INCLUDES:
        - attack_x: x-coordinate of the attack point (Low: -1, High: 1)
        - attack_y: y-coordinate of the attack point (Low: -1, High: 1)
        - defense_x: x-coordinate of the defense point (Low: -1, High: 1)
        - defense_y: y-coordinate of the defense point (Low: -1, High: 1)
        - target_x: x-coordinate of the target point (Low: -1, High: 1)
        - target_y: y-coordinate of the target point (Low: -1, High: 1)
        - defense_attack_theta: angle between the defense and the attack point (low: 0, high: 2*pi)
        - attack_target_theta: angle between the attack and the target point (low: 0, high: 2*pi)
        - defense_angle: angle of the missile (low: 0, high: 2*pi)
        - difference in of defense_attack_theta and defense_angle (low: 0, high: pi)
        - distance_attack_missile: distance between the defense and attack point (low: 0, high: 2.9)
        - distance_attack_target: distance between the attack and target point (low: 0, high: 2.9)
        - defense_attack_theta in sin form (low: -1, high: 1)
        - defense_attack_theta in cos form (low: -1, high: 1)
        - defense_angle in sin form (low: -1, high: 1)
        - defense_angle in cos form (low: -1, high: 1)
        - delta_sin: difference in sin form between defense_attack_theta and defense_angle (low: -2, high: 2)
        - delta_cos: difference in cos form between defense_attack_theta and defense_angle (low: -2, high: 2)
        
        ############################################################
        - NAMING CONVENTION: IF TWO POINTS ARE MENTIONED, DEFENSE ALWAYS GOES FIRST, FOLLOWED BY ATTACK, AND THEN TARGET
        ############################################################

        """
        
        low = np.array([-1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -2, -2])
        high = np.array([1, 1, 1, 1, 1, 1, 2*math.pi, 2*math.pi, 2*math.pi, math.pi, 2.9, 2.9, 1, 1, 1, 1, 2, 2])
        self.observation_space = gym.spaces.Box(low=low, high=high, dtype=np.float32)
        self.radius = 0.02

        self.episode_count = 0
        self.distance_t_minus_one = 0
        self.distance_change = 0

        self.out_of_bounds = 0
        self.interceptions = 0
        self.reached_max_steps = 0
        self.enemy_impacts = 0

        self.defense_positions = []
        self.attack_positions = []
        self.attack_starting_position = 0

        self.max_steps_per_episode = 150
        self.activate_value = 0

    def reset(self, seed=None):
        self.dict_state = {}
        self.activate_enemy_impact = False
        self.defense_positions = []
        self.attack_positions = []
        self.reward = 0
        self.current_step = 0
        self.done = False
        self.create_target()
        self.create_defense(self.target)
        self.create_attack(self.target)

        self.calc_defense_attack_theta(self.defense, self.attack)
        self.calc_attack_target_theta(self.attack, self.target)
        # print("Attack Missile Theta: ", self.defense_attack_theta)
        # print("Attack Target Theta: ", self.attack_target_theta)
        self.initial_defense_angle()
        self.calc_defense_attack_distance()
        self.calc_attack_target_distance()
        self.attack_starting_position = self.attack.copy()

        # self.graph(self.defense, self.attack, self.target)
        self.get_state()

        return self.state, {}

    def create_target(self):
        x = random.uniform(-0.3, 0.3)
        y = random.uniform(-0.3, 0.3)
        self.target = np.array([x, y])

    def create_defense(self, target):
        x = random.uniform((target[0] - self.radius) - 0.15, (target[0] + self.radius) + 0.15)
        y = random.uniform((target[1] - self.radius) - 0.15, (target[1] + self.radius) + 0.15)
        self.defense = np.array([x, y])

    def create_attack(self, target):
        x_side_left = random.uniform(-0.95, max(((target[0] - self.radius) - 0.4), -0.94))
        x_side_right = random.uniform(min(((target[0] + self.radius) + 0.4), 0.94), 0.95)
        y_below = random.uniform(max(((target[1] - self.radius) - 0.4), -0.94), -0.95)
        y_above = random.uniform(max(((target[1] + self.radius) + 0.4), 0.94), 0.95)
        x_inclusive = random.uniform(-0.95, 0.95)
        y_inclusive = random.uniform(-0.95, 0.95)
        y_below_x_inclusive = np.array([x_inclusive, y_below])
        y_above_x_inclusive = np.array([x_inclusive, y_above])
        x_left_y_inclusive = np.array([x_side_left, y_inclusive])
        x_right_y_inclusive = np.array([x_side_right, y_inclusive])

        self.attack = random.choice([y_below_x_inclusive, y_above_x_inclusive, x_left_y_inclusive, x_right_y_inclusive])

    def calc_defense_attack_theta(self, defense, attack):

        # create an adjacent point of the form (attack_x, defense_y)
        adjacent_point = np.array([attack[0], defense[1]])

        # calculate the distance between the adjacent point and the defense, attack points
        adj_point_defense_len = abs(defense[0] - adjacent_point[0]) 
        adj_point_attack_len = abs(attack[1] - adjacent_point[1])

        # calculate the angle, using soh cah toa, where adj_point_defense_len is the adjacent side and adj_point_attack_len is the opposite side
        self.defense_attack_theta = np.arctan(adj_point_attack_len / adj_point_defense_len)
        
        if attack[0] > defense[0]:
            if attack[1] > defense[1]:
                self.defense_attack_theta = self.defense_attack_theta # 1st quadrant
            else: 
                self.defense_attack_theta = (2*math.pi) - self.defense_attack_theta # 360 - theta
        else:
            if attack[1] > defense[1]:
                self.defense_attack_theta = math.pi - self.defense_attack_theta # 180 - theta
            else:
                self.defense_attack_theta = math.pi + self.defense_attack_theta # 180 + theta

    def calc_attack_target_theta(self, attack, target):
        # create an adjacent point of the form (target_x, attack_y)
        adjacent_point = np.array([target[0], attack[1]])

        # calculate the distance between the adjacent point and the attack, target points
        adj_point_attack_len = abs(attack[0] - adjacent_point[0])
        adj_point_target_len = abs(target[1] - adjacent_point[1])
        
        # calculate the angle, using soh cah toa, where adj_point_attack_len is the adjacent side and adj_point_target_len is the opposite side
        self.attack_target_theta = np.arctan(adj_point_target_len / adj_point_attack_len)

        if target[0] > attack[0]:
            if target[1] > attack[1]:
                self.attack_target_theta = self.attack_target_theta
            else:
                self.attack_target_theta = (2*math.pi) - self.attack_target_theta
        else:
            if target[1] > attack[1]:
                self.attack_target_theta = math.pi - self.attack_target_theta
            else:
                self.attack_target_theta = math.pi + self.attack_target_theta        

    def initial_defense_angle(self):
        self.defense_angle = np.random.uniform((self.defense_attack_theta - 2.35619), (self.defense_attack_theta + 2.35619))
        if self.defense_angle > 2*math.pi:
            self.defense_angle = self.defense_angle - 2*math.pi
        elif self.defense_angle < 0:
            self.defense_angle = 2*math.pi + self.defense_angle

    def calculate_distance(self, point1, point2):
        return math.hypot(point1[0] - point2[0], point1[1] - point2[1])
    
    def calc_defense_attack_distance(self):
        self.defense_attack_distance = (self.calculate_distance(self.defense, self.attack) - (2 * self.radius))

    def calc_attack_target_distance(self):
        self.attack_target_distance = (self.calculate_distance(self.attack, self.target) - (2 * self.radius))

    def calc_defense_angle(self, action):
        if action == 0:
            self.defense_angle = self.defense_angle 
        elif action == 1:
            self.defense_angle += 0.174532925
        elif action == 2:
            self.defense_angle -= 0.174532925
        
        if self.defense_angle > 2*math.pi:
            self.defense_angle = self.defense_angle - 2*math.pi
        elif self.defense_angle < 0:
            self.defense_angle = 2*math.pi + self.defense_angle

    def update_coords(self):
        self.defense[0] += (0.02 * math.cos(self.defense_angle)) # gotta test this
        self.defense[1] += (0.02 * math.sin(self.defense_angle))

        # print("Attack: ", self.attack)
        # print("Attack theta: ", self.defense_attack_theta)
        # print("0.02 * math.cos(self.defense_attack_theta): ", 0.02 * math.cos(self.defense_attack_theta))
        # print("0.02 * math.sin(self.defense_attack_theta): ", 0.02 * math.sin(self.defense_attack_theta))

        self.attack[0] += (0.02 * math.cos(self.attack_target_theta))
        self.attack[1] += (0.02 * math.sin(self.attack_target_theta))

        # print("Attack after: ", self.attack)

        # self.attack[0] = self.attack_starting_position[0] + (0.02 * math.cos(self.defense_attack_theta))
        # self.attack[1] = (0.02 * math.sin(self.defense_attack_theta))

        self.defense_positions.append(self.defense.copy())
        self.attack_positions.append(self.attack.copy())

    def calculate_reward(self):
        self.calc_defense_attack_distance()
        self.calc_attack_target_distance()

        if self.attack_target_distance < 0:
            print("ENEMY HIT!")
            self.activate_enemy_impact = True
            self.activate_value = 0
            self.dict_state = self.get_state_dict()
            self.reward = -10000
            self.done = True
            self.enemy_impacts += 1

        elif self.defense_attack_distance < 0:
            print("HIT!")
            self.activate_interception = True
            self.reward = 10000
            self.done = True
            self.interceptions += 1
        else:
            self.angle_diff = abs(self.defense_attack_theta - self.defense_angle)
            self.angle_diff = min(self.angle_diff, 2*math.pi - self.angle_diff)
            self.reward = 1/self.angle_diff
                
        if self.defense[0] < -1 or self.defense[0] > 1 or self.defense[1] < -1 or self.defense[1] > 1:
            print("OUT OF BOUNDS")
            self.reward = -1000
            self.done = True
            self.out_of_bounds += 1

    def angle_conversion(self):
        self.sin_defense_attack_theta, self.sin_defense_angle = np.sin(self.defense_attack_theta), np.sin(self.defense_angle)
        self.cos_defense_attack_theta, self.cos_defense_angle = np.cos(self.defense_attack_theta), np.cos(self.defense_angle)

        self.delta_sin = self.sin_defense_attack_theta - self.sin_defense_angle
        self.delta_cos = self.cos_defense_attack_theta - self.cos_defense_angle

        """STATE INCLUDES:
        - attack_x: x-coordinate of the attack point (Low: -1, High: 1)
        - attack_y: y-coordinate of the attack point (Low: -1, High: 1)
        - defense_x: x-coordinate of the defense point (Low: -1, High: 1)
        - defense_y: y-coordinate of the defense point (Low: -1, High: 1)
        - target_x: x-coordinate of the target point (Low: -1, High: 1)
        - target_y: y-coordinate of the target point (Low: -1, High: 1)
        - defense_attack_theta: angle between the defense and the attack point (low: 0, high: 2*pi)
        - attack_target_theta: angle between the attack and the target point (low: 0, high: 2*pi)
        - defense_angle: angle of the missile (low: 0, high: 2*pi)
        - difference in of defense_attack_theta and defense_angle (low: 0, high: pi)
        - distance_attack_missile: distance between the defense and attack point (low: 0, high: 2.9)
        - distance_attack_target: distance between the attack and target point (low: 0, high: 2.9)
        - defense_attack_theta in sin form (low: -1, high: 1)
        - defense_attack_theta in cos form (low: -1, high: 1)
        - defense_angle in sin form (low: -1, high: 1)
        - defense_angle in cos form (low: -1, high: 1)
        - delta_sin: difference in sin form between defense_attack_theta and defense_angle (low: -2, high: 2)
        - delta_cos: difference in cos form between defense_attack_theta and defense_angle (low: -2, high: 2)
        """

    def get_state(self):
        self.angle_conversion()

        self.state = np.array([
            self.attack[0], self.attack[1], 
            self.defense[0], self.defense[1], 
            self.target[0], self.target[1],
            self.defense_attack_theta, self.attack_target_theta,
            self.defense_angle,
            min(abs(self.defense_attack_theta - self.defense_angle), 2*math.pi - abs(self.defense_attack_theta - self.defense_angle)),
            self.defense_attack_distance,
            self.attack_target_distance,
            self.sin_defense_attack_theta, self.cos_defense_attack_theta, 
            self.sin_defense_angle, self.cos_defense_angle,
            self.delta_sin, self.delta_cos
        ])

    def get_state_dict(self):
        return {
            "self.activate": self.activate_enemy_impact,
            "attack_x": self.attack[0],
            "attack_y": self.attack[1],
            "defense_x": self.defense[0],
            "defense_y": self.defense[1],
            "target_x": self.target[0],
            "target_y": self.target[1],
            "defense_attack_theta": self.defense_attack_theta,
            "attack_target_theta": self.attack_target_theta,
            "defense_angle": self.defense_angle,
            "angle_diff": min(abs(self.defense_attack_theta - self.defense_angle), 2*math.pi - abs(self.defense_attack_theta - self.defense_angle)),
            "distance_attack_missile": self.defense_attack_distance,
            "distance_attack_target": self.attack_target_distance,
            "sin_defense_attack_theta": self.sin_defense_attack_theta,
            "cos_defense_attack_theta": self.cos_defense_attack_theta,
            "sin_defense_angle": self.sin_defense_angle,
            "cos_defense_angle": self.cos_defense_angle,
            "delta_sin": self.delta_sin,
            "delta_cos": self.delta_cos
        }
                 
    def step(self, action):
        # print("............................................")
        # print("BEFORE ACTION")
        # print("ATTACK_TARGET_THETA: ", self.attack_target_theta)
        # print("DEFENSE_ATTACK_THETA: ", self.defense_attack_theta)
        # print("DEFENSE_ANGLE: ", self.defense_angle)
        # print("DEFENSE: ", self.defense)
        # print("ATTACK: ", self.attack)
        # print("TARGET: ", self.target)
        # print("............................................")

        self.distance_t_minus_one = self.defense_attack_distance
        self.calc_defense_angle(action)
        self.update_coords()
        self.calc_defense_attack_theta(self.defense, self.attack)
        self.calculate_reward()
        self.current_step += 1

        if self.current_step >= self.max_steps_per_episode:
            print("MAX STEPS REACHED")
            self.done = True
            self.reward = -1000
            self.reached_max_steps += 1

        self.get_state()
        # print("--------------------------------------------")
        # print("AFTER ACTION")
        # print("ATTACK_TARGET_THETA: ", self.attack_target_theta)
        # print("DEFENSE_ATTACK_THETA: ", self.defense_attack_theta)
        # print("DEFENSE_ANGLE: ", self.defense_angle)
        # print("DEFENSE: ", self.defense)
        # print("ATTACK: ", self.attack)
        # print("TARGET: ", self.target)
        # print("REWARD: ", self.reward)
        # print("--------------------------------------------")
        # self.graph(self.defense, self.attack, self.target)
        return self.state, self.reward, self.done, False, {'activated': self.activate_enemy_impact}

    def graph(self, defense, attack, target):
        fig, ax = plt.subplots()
        plt.xlim(-1, 1)
        plt.ylim(-1, 1)

        plt.axhline(0, color='black', linewidth=0.5)
        plt.axvline(0, color='black', linewidth=0.5)
        plt.grid(True)

        # Plot trails
        if self.defense_positions:
            defense_xs, defense_ys = zip(*self.defense_positions)
            ax.plot(defense_xs, defense_ys, color='#858585', label='Defense Trail')  # Blue line for defense

        if self.attack_positions:
            attack_xs, attack_ys = zip(*self.attack_positions)
            ax.plot(attack_xs, attack_ys, color='#FFA281', label='Attack Trail')  # Red line for attack

        # Plot current positions
        plt.scatter(defense[0], defense[1], color='#1C1C1C')
        plt.scatter(attack[0], attack[1], color='#FF5A1F')
        plt.scatter(self.target[0], self.target[1], color='#85A3FF')

        ax.set_aspect('equal')
        plt.show()

In [2]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML, display

def animate_episode(episode_data, save_name):
    episode_past_defense_positions = episode_data["past_defense_positions"]
    past_defense_x, past_defense_y = zip(*episode_past_defense_positions)
    episode_past_attack_positions = episode_data["past_attack_positions"]
    past_attack_x, past_attack_y = zip(*episode_past_attack_positions)
    episode_attack_positions = episode_data["attack_positions"]
    attack_x, attack_y = zip(*episode_attack_positions)
    episode_defense_positions = episode_data["defense_positions"]
    defense_x, defense_y = zip(*episode_defense_positions)
    target_position = episode_data["target_position"]

    # Create figure and axis
    fig, ax = plt.subplots()
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)
    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(True)

    trail_attack, = ax.plot([], [], color='#FFA281', label='Attack Trail')   # Light Red (ATTACK)
    trail_defense, = ax.plot([], [], color='#858585', label='Defense Trail')  
    scatter1, = ax.plot([], [], linestyle='', marker='o', color='#1C1C1C', label='Defense Position')  
    scatter2, = ax.plot([], [], linestyle='', marker='o', color='#FF5A1F', label='Attack Position')  # Black circles
    scatter3, = ax.plot([], [], linestyle='', marker='o', color='#85A3FF', label='Target Position')  # Light Blue circles

    # Update function for animation
    def update(frame):
        trail_attack.set_data(past_attack_x[:frame+1], past_attack_y[:frame+1])
        trail_defense.set_data(past_defense_x[:frame+1], past_defense_y[:frame+1])
        
        scatter1.set_data(attack_x[frame], attack_y[frame])
        scatter2.set_data(defense_x[frame], defense_y[frame])
        scatter3.set_data(target_position[0], target_position[1])
        return trail_attack, trail_defense, scatter1, scatter2, scatter3

    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=len(past_defense_x), interval=200, blit=True)

    # Add legend outside the plot
    ax.legend()

    # # Option 1: Save animation
    # ani.save(save_name, writer='pillow')    
    # print(f"Animation saved as {save_name}")

    # Option 2: Display using HTML
    plt.close(fig)
    video = ani.to_html5_video()
    html = display(HTML(video))

    plt.show()


In [ ]:
from copy import deepcopy

log_number = 1

def run_episode(env, model):
    # If you have only one environment wrapped, you can directly access it
    single_env = env.envs[0]
    obs = env.reset()
    done = False
    animate = False
    total_reward = 0
    step = 0
    episode_data = None
    global log_number

    print("EPISODE NUMBER:", log_number)

    episode_data = {
    'past_defense_positions': [], # In upgraded code, we'd only need defense_positions, this is slop, let it work for now
    'past_attack_positions': [],
    'attack_positions': [],
    'defense_positions': [], # because you're basically storing the same data in two different places
    'target_position': [],
    'actions': [],
    'rewards': [],
    'defense_angle': [],
    'defense_attack_theta': [],
    }

    episode_data['target_position'] = obs[0][4:6]

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        episode_data['actions'].append(deepcopy(action))
        episode_data['past_defense_positions'].append(deepcopy(np.array([obs[0][2], obs[0][3]])))
        episode_data['past_attack_positions'].append(deepcopy(np.array([obs[0][0], obs[0][1]])))
        obs, reward, done, info = env.step(action)
        episode_data['rewards'].append(deepcopy(reward))
        episode_data['defense_angle'].append(deepcopy(obs[0][8]))
        episode_data['defense_attack_theta'].append(deepcopy(obs[0][6]))
        episode_data['attack_positions'].append(deepcopy(np.array([obs[0][2], obs[0][3]])))
        episode_data['defense_positions'].append(deepcopy(np.array([obs[0][0], obs[0][1]])))
        total_reward += reward
        step += 1

    print(f"Episode finished in {step} steps with reward {total_reward}. Out of bounds: {single_env.out_of_bounds}, Interceptions: {single_env.interceptions}, Reached max steps: {single_env.reached_max_steps}, Enemy impacts: {single_env.enemy_impacts}")

    log_number += 1
    
    if episode_data is not None:
        episode_file_name = f'episode_{log_number}_(2.4.2.2)_PROD_8.gif'
        animate_episode(episode_data, episode_file_name)
        #print defense_angles
        print("DEFENSE ANGLES: ", episode_data['defense_angle'])
        print("DEFENSE ATTACK THETAS: ", episode_data['defense_attack_theta'])
        print("ACTIONS: ", episode_data['actions'])
        print("REWARDS: ", episode_data['rewards'])

    # Now access the specific attributes from the single_env which is your actual missile_interception instance
    return step, total_reward, single_env.out_of_bounds, single_env.interceptions, single_env.reached_max_steps, single_env.enemy_impacts

# Create the environment
env = make_vec_env(lambda: missile_interception(), n_envs=1)

# Create the model
model = DQN("MlpPolicy", env, verbose=1, tensorboard_log="./dqn_missile_guidance_local_v(2.4.2.2)_PROD8")

# Create a summary writer
summary_writer = tf.summary.create_file_writer('./dqn_missile_guidance_local_v(2.4.2.2)_PROD8/custom_metrics')

# Training loop
total_timesteps = 1000000
eval_interval = 100000  # Evaluate and log every 10000 steps
for step in range(0, total_timesteps, eval_interval):

    # Train for a number of timesteps
    model.learn(total_timesteps=eval_interval, reset_num_timesteps=False)
    
    # Run an evaluation episode
    episode_length, episode_reward, out_of_bounds, interceptions, reached_max_steps, enemy_impacts = run_episode(env, model)
    
    # Log the results
    with summary_writer.as_default():
        tf.summary.scalar('Evaluation/Episode Length', episode_length, step=step)
        # Ensure episode_reward and other metrics are scalars by using .item() if they are numpy arrays or tensors
        tf.summary.scalar('Evaluation/Episode Reward', episode_reward.item() if isinstance(episode_reward, np.ndarray) else episode_reward, step=step)
        tf.summary.scalar('Evaluation/Out of Bounds Count', out_of_bounds.item() if isinstance(out_of_bounds, np.ndarray) else out_of_bounds, step=step)
        tf.summary.scalar('Evaluation/Interceptions Count', interceptions.item() if isinstance(interceptions, np.ndarray) else interceptions, step=step)
        tf.summary.scalar('Evaluation/Reached Max Steps Count', reached_max_steps.item() if isinstance(reached_max_steps, np.ndarray) else reached_max_steps, step=step)
        tf.summary.scalar('Evaluation/Enemy Impacts Count', enemy_impacts.item() if isinstance(enemy_impacts, np.ndarray) else enemy_impacts, step=step)
        summary_writer.flush()


# Save the final model
model.save("dqn_missile_guidance_v(2.4.2.2)_moving")

In [ ]:
# from copy import deepcopy

# # loop through three episodes and animate them
# for episode in range(3):
#     env = missile_interception()
#     episode_data = {
#     'past_defense_positions': [], # In upgraded code, we'd only need defense_positions, this is slop, let it work for now
#     'past_attack_positions': [],
#     'attack_positions': [],
#     'defense_positions': [], # because you're basically storing the same data in two different places
#     'target_position': [],
#     }

#     obs = env.reset()
#     obs = obs[0]
#     episode_data['target_position'] = obs[4:6]
#     print("Episode: ", obs)
#     done = False
#     while not done:
#         action = env.action_space.sample()
#         episode_data['past_defense_positions'].append(deepcopy(np.array([obs[2], obs[3]])))
#         episode_data['past_attack_positions'].append(deepcopy(np.array([obs[0], obs[1]])))
#         obs, reward, done, truncated, info = env.step(action)
#         print(obs)
#         print("AHHHHHHHHH", np.array([obs[2]]))
#         print("ogh", obs[3])
#         episode_data['attack_positions'].append(deepcopy(np.array([obs[2], obs[3]])))
#         episode_data['defense_positions'].append(deepcopy(np.array([obs[0], obs[1]])))

#     animate_episode(episode_data, f"episode_{episode}_PITOFINAL.gif")

In [ ]:
# # ---------------------------------------------------------------------------------------------------------------------------------------------------------
# # .........................................................................................................................................................
# # ---------------------------------------------------------------------------------------------------------------------------------------------------------
# # COPY OF THE CODE ABOVE: THIS VERSION IS FOR TESTING THE ANIMATION FUNCTION 

# from copy import deepcopy

# log_number = 1

# def run_episode(env, model):
#     # If you have only one environment wrapped, you can directly access it
#     single_env = env.envs[0]
#     obs = env.reset()
#     done = False
#     animate = False
#     total_reward = 0
#     step = 0
#     episode_data = None
#     global log_number

#     print("EPISODE NUMBER:", log_number)

#     episode_data = {
#     'past_defense_positions': [], # In upgraded code, we'd only need defense_positions, this is slop, let it work for now
#     'attack_positions': [],
#     'defense_positions': [], # because you're basically storing the same data in two different places
#     'theta': [],
#     'missile_angle': [],
#     'actions': [],
#     }

#     while not done:
#         action, _ = model.predict(obs, deterministic=True)
#         episode_data['actions'].append(deepcopy(action))
#         episode_data['past_defense_positions'].append(deepcopy(np.array([obs[0][2], obs[0][3]])))
#         obs, reward, done, info = env.step(action)
#         episode_data['attack_positions'].append(deepcopy(np.array([obs[0][2], obs[0][3]])))
#         episode_data['defense_positions'].append(deepcopy(np.array([obs[0][0], obs[0][1]])))
#         episode_data['theta'].append(deepcopy(obs[0][4]))
#         episode_data['missile_angle'].append(deepcopy(obs[0][5]))
#         total_reward += reward
#         step += 1

#     print(f"Episode finished in {step} steps with reward {total_reward}. Out of bounds: {single_env.out_of_bounds}, Interceptions: {single_env.interceptions}, Reached max steps: {single_env.reached_max_steps}")

#     log_number += 1
    
#     if episode_data is not None:
#         episode_file_name = f'episode_{log_number}_v9_dqn_bone_angle.gif'
#         animate_episode(episode_data, episode_file_name)

#     # Now access the specific attributes from the single_env which is your actual missile_interception instance
#     return step, total_reward, single_env.out_of_bounds, single_env.interceptions, single_env.reached_max_steps

# # Create the environment
# env = make_vec_env(lambda: missile_interception(), n_envs=1)

# # Create the model
# model = DQN("MlpPolicy", env, verbose=1, tensorboard_log="./dqn_missile_guidance_local_v9_bone_angle")

# # Create a summary writer
# summary_writer = tf.summary.create_file_writer('./dqn_missile_guidance_local_v9_bone_angle/custom_metrics')

# # Training loop
# total_timesteps = 1000000
# eval_interval = 100000  # Evaluate and log every 10000 steps

# for step in range(0, total_timesteps, eval_interval):
#     # Train for a number of timesteps
#     model.learn(total_timesteps=eval_interval, reset_num_timesteps=False)
    
#     # Run an evaluation episode
#     episode_length, episode_reward, out_of_bounds, interceptions, reached_max_steps = run_episode(env, model)
    
#     # Log the results
#     with summary_writer.as_default():
#         tf.summary.scalar('Evaluation/Episode Length', episode_length, step=step)
#         # Ensure episode_reward and other metrics are scalars by using .item() if they are numpy arrays or tensors
#         tf.summary.scalar('Evaluation/Episode Reward', episode_reward.item() if isinstance(episode_reward, np.ndarray) else episode_reward, step=step)
#         tf.summary.scalar('Evaluation/Out of Bounds Count', out_of_bounds.item() if isinstance(out_of_bounds, np.ndarray) else out_of_bounds, step=step)
#         tf.summary.scalar('Evaluation/Interceptions Count', interceptions.item() if isinstance(interceptions, np.ndarray) else interceptions, step=step)
#         tf.summary.scalar('Evaluation/Reached Max Steps Count', reached_max_steps.item() if isinstance(reached_max_steps, np.ndarray) else reached_max_steps, step=step)
#         summary_writer.flush()


# # Save the final model
# model.save("dqn_missile_guidance_v9_bone_angle")

In [4]:
import numpy as np
from stable_baselines3 import DQN
from copy import deepcopy
import tensorflow as tf
import gymnasium as gym
import matplotlib.pyplot as plt
from stable_baselines3.common.env_util import make_vec_env

# Function to run an episode
def run_episode(env, model):
    single_env = env.envs[0]
    obs = env.reset()
    done = False
    total_reward = 0
    step = 0
    episode_data = {
        'past_defense_positions': [],
        'past_attack_positions': [],
        'attack_positions': [],
        'defense_positions': [],
        'target_position': [],
        'actions': [],
        'rewards': [],
        'defense_angle': [],
        'defense_attack_theta': [],
    }
    
    episode_data['target_position'] = obs[0][4:6]

    while not done:
        action, _ = model.predict(obs, deterministic=True)
        episode_data['actions'].append(deepcopy(action))
        episode_data['past_defense_positions'].append(deepcopy(np.array([obs[0][2], obs[0][3]])))
        episode_data['past_attack_positions'].append(deepcopy(np.array([obs[0][0], obs[0][1]])))
        obs, reward, done, info = env.step(action)
        episode_data['rewards'].append(deepcopy(reward))
        episode_data['defense_angle'].append(deepcopy(obs[0][8]))
        episode_data['defense_attack_theta'].append(deepcopy(obs[0][6]))
        episode_data['attack_positions'].append(deepcopy(np.array([obs[0][2], obs[0][3]])))
        episode_data['defense_positions'].append(deepcopy(np.array([obs[0][0], obs[0][1]])))
        total_reward += reward
        step += 1

    print(f"Episode finished in {step} steps with reward {total_reward}. Out of bounds: {single_env.out_of_bounds}, Interceptions: {single_env.interceptions}, Reached max steps: {single_env.reached_max_steps}, Enemy impacts: {single_env.enemy_impacts}")

    return step, total_reward, single_env.out_of_bounds, single_env.interceptions, single_env.reached_max_steps, single_env.enemy_impacts, info, episode_data

# Function to evaluate the model
def evaluate_model(model_path, n_episodes=100):
    # Create the environment
    env = make_vec_env(lambda: missile_interception(), n_envs=1)
    
    # Load the trained model
    model = DQN.load(model_path)
    
    target_hit_episodes = []
    
    # Run multiple evaluation episodes
    for i in range(n_episodes):
        print(f"Running episode {i + 1}")
        episode_length, episode_reward, out_of_bounds, interceptions, reached_max_steps, enemy_impacts, info, episode_data = run_episode(env, model)

        print(info[0]['activated'])
        print("........................................................................................")

        if info[0]['activated'] == True:
            target_hit_episodes.append(episode_data)
    
    return target_hit_episodes

# Function to graph the episode data
def graph_episode(defense_positions, attack_positions, target_position):
    fig, ax = plt.subplots()
    plt.xlim(-1, 1)
    plt.ylim(-1, 1)

    plt.axhline(0, color='black', linewidth=0.5)
    plt.axvline(0, color='black', linewidth=0.5)
    plt.grid(True)

    # Plot trails
    if defense_positions:
        defense_xs, defense_ys = zip(*defense_positions)
        ax.plot(defense_xs, defense_ys, color='#858585', label='Defense Trail')  # Blue line for defense

    if attack_positions:
        attack_xs, attack_ys = zip(*attack_positions)
        ax.plot(attack_xs, attack_ys, color='#FFA281', label='Attack Trail')  # Red line for attack

    # Plot current positions
    plt.scatter(defense_positions[-1][0], defense_positions[-1][1], color='#1C1C1C', label='Defense Position')
    plt.scatter(attack_positions[-1][0], attack_positions[-1][1], color='#FF5A1F', label='Attack Position')
    plt.scatter(target_position[0], target_position[1], color='#85A3FF', label='Target Position')

    ax.set_aspect('equal')
    plt.legend()
    plt.show()

# Path to the saved model
model_path = "dqn_missile_guidance_v(2.4.2.2)_moving"

# Evaluate the model and get episodes with target hits
target_hit_episodes = evaluate_model(model_path, n_episodes=200)

print("HEREEEEEEEEEEEEEEEEEEE")

# Graph the episodes with target hits
i = 0
for episode_data in target_hit_episodes:
    print("INISDE HERE")
    episode_name = f'Episode {i} collision.gif '
    animate_episode(episode_data, episode_name)
    i += 1


Running episode 1
HIT!
Episode finished in 31 steps with reward [11107.487]. Out of bounds: 0, Interceptions: 1, Reached max steps: 0, Enemy impacts: 0
False
........................................................................................
Running episode 2
HIT!
Episode finished in 28 steps with reward [10635.368]. Out of bounds: 0, Interceptions: 2, Reached max steps: 0, Enemy impacts: 0
False
........................................................................................
Running episode 3
HIT!
Episode finished in 10 steps with reward [10298.881]. Out of bounds: 0, Interceptions: 3, Reached max steps: 0, Enemy impacts: 0
False
........................................................................................
Running episode 4
HIT!
Episode finished in 25 steps with reward [14601.524]. Out of bounds: 0, Interceptions: 4, Reached max steps: 0, Enemy impacts: 0
False
........................................................................................
Running epis

c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.out_of_bounds to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.out_of_bounds` for environment variables or `env.get_wrapper_attr('out_of_bounds')` that will search the reminding wrappers.
  logger.warn(
c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.interceptions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.interceptions` for environment variables or `env.get_wrapper_attr('interceptions')` that will search the reminding wrappers.
  logger.warn(
c:\Users\thoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.reached_max_steps to get variables from other wrappers is deprecated and will be 

HIT!
Episode finished in 17 steps with reward [10174.938]. Out of bounds: 0, Interceptions: 21, Reached max steps: 0, Enemy impacts: 9
False
........................................................................................
Running episode 31
HIT!
Episode finished in 16 steps with reward [10794.953]. Out of bounds: 0, Interceptions: 22, Reached max steps: 0, Enemy impacts: 9
False
........................................................................................
Running episode 32
ENEMY HIT!
ACTIVATE ENEMY IMPACT True
state {'self.activate': True, 'attack_x': 0.07067428389002722, 'attack_y': 0.1018836067871075, 'defense_x': -0.06241452770754753, 'defense_y': 0.24756897694025873, 'target_x': 0.0946741079393158, 'target_y': 0.08423818423681745, 'defense_attack_theta': 5.452632330551299, 'attack_target_theta': 5.64920334268021, 'defense_angle': 4.554743455265155, 'angle_diff': 0.897888875286144, 'distance_attack_missile': 0.15732424800088668, 'distance_attack_target': -0.01021

INISDE HERE


INISDE HERE


...

INISDE HERE


INISDE HERE


INISDE HERE


INISDE HERE


INISDE HERE
